In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict

from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import LabelEncoder

## Naive Bayes krok po kroku z `pandas`

In [2]:
# KROK 1: Utworzenie danych
data = {
    'Pogoda': ['Słońce', 'Słońce', 'Pochmurno', 'Deszcz', 'Deszcz',
               'Deszcz', 'Pochmurno', 'Słońce', 'Słońce', 'Deszcz',
               'Słońce', 'Pochmurno', 'Pochmurno', 'Deszcz'],
    'Temperatura': ['Ciepło', 'Ciepło', 'Ciepło', 'Umiarkowana', 'Zimno',
                    'Zimno', 'Zimno', 'Umiarkowana', 'Zimno', 'Umiarkowana',
                    'Umiarkowana', 'Umiarkowana', 'Ciepło', 'Umiarkowana'],
    'Wiatr': ['Słaby', 'Silny', 'Słaby', 'Słaby', 'Słaby',
              'Silny', 'Silny', 'Słaby', 'Słaby', 'Słaby',
              'Silny', 'Silny', 'Słaby', 'Silny'],
    'Gra': ['TAK', 'TAK', 'TAK', 'TAK', 'TAK',
            'NIE', 'TAK', 'NIE', 'TAK', 'TAK',
            'TAK', 'TAK', 'TAK', 'NIE']
}

df = pd.DataFrame(data)

In [3]:
print(df.to_string(index=True))

       Pogoda  Temperatura  Wiatr  Gra
0      Słońce       Ciepło  Słaby  TAK
1      Słońce       Ciepło  Silny  TAK
2   Pochmurno       Ciepło  Słaby  TAK
3      Deszcz  Umiarkowana  Słaby  TAK
4      Deszcz        Zimno  Słaby  TAK
5      Deszcz        Zimno  Silny  NIE
6   Pochmurno        Zimno  Silny  TAK
7      Słońce  Umiarkowana  Słaby  NIE
8      Słońce        Zimno  Słaby  TAK
9      Deszcz  Umiarkowana  Słaby  TAK
10     Słońce  Umiarkowana  Silny  TAK
11  Pochmurno  Umiarkowana  Silny  TAK
12  Pochmurno       Ciepło  Słaby  TAK
13     Deszcz  Umiarkowana  Silny  NIE


In [4]:
print("=" * 60)
print("KROK 1: PRAWDOPODOBIEŃSTWA A PRIORI")
print("=" * 60)
class_counts = df['Gra'].value_counts()
total = len(df)
priors = class_counts / total

print(f"Liczba przypadków TAK: {class_counts['TAK']}")
print(f"Liczba przypadków NIE: {class_counts['NIE']}")
print(f"Razem: {total}")
print()
print(f"P(Gra = TAK) = {class_counts['TAK']}/{total} = {priors['TAK']:.3f}")
print(f"P(Gra = NIE) = {class_counts['NIE']}/{total} = {priors['NIE']:.3f}")
print()

KROK 1: PRAWDOPODOBIEŃSTWA A PRIORI
Liczba przypadków TAK: 11
Liczba przypadków NIE: 3
Razem: 14

P(Gra = TAK) = 11/14 = 0.786
P(Gra = NIE) = 3/14 = 0.214



In [5]:
# KROK 2: Obliczenie prawdopodobieństw a priori
print("=" * 60)
print("KROK 1: PRAWDOPODOBIEŃSTWA A PRIORI")
print("=" * 60)
class_counts = df['Gra'].value_counts()
total = len(df)
priors = class_counts / total

print(f"Liczba przypadków TAK: {class_counts['TAK']}")
print(f"Liczba przypadków NIE: {class_counts['NIE']}")
print(f"Razem: {total}")
print()
print(f"P(Gra = TAK) = {class_counts['TAK']}/{total} = {priors['TAK']:.3f}")
print(f"P(Gra = NIE) = {class_counts['NIE']}/{total} = {priors['NIE']:.3f}")
print()

KROK 1: PRAWDOPODOBIEŃSTWA A PRIORI
Liczba przypadków TAK: 11
Liczba przypadków NIE: 3
Razem: 14

P(Gra = TAK) = 11/14 = 0.786
P(Gra = NIE) = 3/14 = 0.214



In [6]:
# KROK 3: Obliczenie prawdopodobieństw warunkowych
print("=" * 60)
print("KROK 2: PRAWDOPODOBIEŃSTWA WARUNKOWE")
print("=" * 60)

features = ['Pogoda', 'Temperatura', 'Wiatr']
conditionals = {}

for feature in features:
    print(f"\n{'='*60}")
    print(f"Cecha: {feature.upper()}")
    print(f"{'='*60}")
    conditionals[feature] = {}

    for class_val in ['TAK', 'NIE']:
        class_data = df[df['Gra'] == class_val]
        class_count = len(class_data)

        print(f"\nDla klasy {class_val} ({class_count} przypadków):")
        print("-" * 40)

        conditionals[feature][class_val] = {}
        feature_counts = class_data[feature].value_counts()

        for feature_val in df[feature].unique():
            count = feature_counts.get(feature_val, 0)
            prob = count / class_count
            conditionals[feature][class_val][feature_val] = prob

            print(f"  {feature_val:12s}: {count} przypadków → "
                  f"P({feature}={feature_val} | {class_val}) = "
                  f"{count}/{class_count} = {prob:.3f}")

KROK 2: PRAWDOPODOBIEŃSTWA WARUNKOWE

Cecha: POGODA

Dla klasy TAK (11 przypadków):
----------------------------------------
  Słońce      : 4 przypadków → P(Pogoda=Słońce | TAK) = 4/11 = 0.364
  Pochmurno   : 4 przypadków → P(Pogoda=Pochmurno | TAK) = 4/11 = 0.364
  Deszcz      : 3 przypadków → P(Pogoda=Deszcz | TAK) = 3/11 = 0.273

Dla klasy NIE (3 przypadków):
----------------------------------------
  Słońce      : 1 przypadków → P(Pogoda=Słońce | NIE) = 1/3 = 0.333
  Pochmurno   : 0 przypadków → P(Pogoda=Pochmurno | NIE) = 0/3 = 0.000
  Deszcz      : 2 przypadków → P(Pogoda=Deszcz | NIE) = 2/3 = 0.667

Cecha: TEMPERATURA

Dla klasy TAK (11 przypadków):
----------------------------------------
  Ciepło      : 4 przypadków → P(Temperatura=Ciepło | TAK) = 4/11 = 0.364
  Umiarkowana : 4 przypadków → P(Temperatura=Umiarkowana | TAK) = 4/11 = 0.364
  Zimno       : 3 przypadków → P(Temperatura=Zimno | TAK) = 3/11 = 0.273

Dla klasy NIE (3 przypadków):
------------------------------------

In [7]:
# KROK 4: Wyświetlenie tabel prawdopodobieństw
print("\n" + "=" * 60)
print("PODSUMOWANIE - TABELE PRAWDOPODOBIEŃSTW")
print("=" * 60)

for feature in features:
    print(f"\n{feature.upper()}:")
    print("-" * 60)
    print(f"{'Wartość':15s} | P({feature}|TAK) | P({feature}|NIE)")
    print("-" * 60)

    for val in df[feature].unique():
        prob_tak = conditionals[feature]['TAK'].get(val, 0)
        prob_nie = conditionals[feature]['NIE'].get(val, 0)
        print(f"{val:15s} | {prob_tak:13.3f} | {prob_nie:13.3f}")


PODSUMOWANIE - TABELE PRAWDOPODOBIEŃSTW

POGODA:
------------------------------------------------------------
Wartość         | P(Pogoda|TAK) | P(Pogoda|NIE)
------------------------------------------------------------
Słońce          |         0.364 |         0.333
Pochmurno       |         0.364 |         0.000
Deszcz          |         0.273 |         0.667

TEMPERATURA:
------------------------------------------------------------
Wartość         | P(Temperatura|TAK) | P(Temperatura|NIE)
------------------------------------------------------------
Ciepło          |         0.364 |         0.000
Umiarkowana     |         0.364 |         0.667
Zimno           |         0.273 |         0.333

WIATR:
------------------------------------------------------------
Wartość         | P(Wiatr|TAK) | P(Wiatr|NIE)
------------------------------------------------------------
Słaby           |         0.636 |         0.333
Silny           |         0.364 |         0.667


In [8]:
# KROK 5: PREDYKCJA dla nowego rekordu
print("\n" + "=" * 60)
print("KROK 3: PREDYKCJA DLA NOWEGO REKORDU")
print("=" * 60)

new_record = {
    'Pogoda': 'Słońce',
    'Temperatura': 'Zimno',
    'Wiatr': 'Silny'
}

print(f"\nNowy rekord:")
for key, val in new_record.items():
    print(f"  {key}: {val}")
print()

# Obliczenia dla każdej klasy
scores = {}

for class_val in ['TAK', 'NIE']:
    print(f"\n{'='*60}")
    print(f"Obliczenia dla klasy: {class_val}")
    print(f"{'='*60}")

    # Zaczynamy od prior
    score = priors[class_val]
    print(f"\n1. Prior: P({class_val}) = {score:.3f}")

    # Mnożymy przez każdą cechę
    step = 2
    for feature, value in new_record.items():
        prob = conditionals[feature][class_val].get(value, 0)
        print(f"{step}. P({feature}={value} | {class_val}) = {prob:.3f}")
        score *= prob
        print(f"   Score po mnożeniu: {score:.6f}")
        step += 1

    print(f"\nWynik końcowy dla {class_val}: {score:.6f}")
    scores[class_val] = score


KROK 3: PREDYKCJA DLA NOWEGO REKORDU

Nowy rekord:
  Pogoda: Słońce
  Temperatura: Zimno
  Wiatr: Silny


Obliczenia dla klasy: TAK

1. Prior: P(TAK) = 0.786
2. P(Pogoda=Słońce | TAK) = 0.364
   Score po mnożeniu: 0.285714
3. P(Temperatura=Zimno | TAK) = 0.273
   Score po mnożeniu: 0.077922
4. P(Wiatr=Silny | TAK) = 0.364
   Score po mnożeniu: 0.028335

Wynik końcowy dla TAK: 0.028335

Obliczenia dla klasy: NIE

1. Prior: P(NIE) = 0.214
2. P(Pogoda=Słońce | NIE) = 0.333
   Score po mnożeniu: 0.071429
3. P(Temperatura=Zimno | NIE) = 0.333
   Score po mnożeniu: 0.023810
4. P(Wiatr=Silny | NIE) = 0.667
   Score po mnożeniu: 0.015873

Wynik końcowy dla NIE: 0.015873


In [9]:
# Normalizacja
print("\n" + "=" * 60)
print("NORMALIZACJA I WYNIK KOŃCOWY")
print("=" * 60)

total_score = sum(scores.values())
print(f"\nSuma scores: {scores['TAK']:.6f} + {scores['NIE']:.6f} = {total_score:.6f}")
print()

normalized = {k: v/total_score for k, v in scores.items()}

print("Prawdopodobieństwa po normalizacji:")
for class_val in ['TAK', 'NIE']:
    percentage = normalized[class_val] * 100
    print(f"  P({class_val} | cechy) = {scores[class_val]:.6f} / {total_score:.6f} = "
          f"{normalized[class_val]:.3f} = {percentage:.1f}%")

print("\n" + "=" * 60)
prediction = max(normalized, key=normalized.get)
print(f"PREDYKCJA: {prediction}")
print(f"Pewność: {normalized[prediction]*100:.1f}%")
print("=" * 60)


NORMALIZACJA I WYNIK KOŃCOWY

Suma scores: 0.028335 + 0.015873 = 0.044208

Prawdopodobieństwa po normalizacji:
  P(TAK | cechy) = 0.028335 / 0.044208 = 0.641 = 64.1%
  P(NIE | cechy) = 0.015873 / 0.044208 = 0.359 = 35.9%

PREDYKCJA: TAK
Pewność: 64.1%


# Obliczenia z sklearn

In [11]:
# Kodowanie cech
le_dict = {}
features = ['Pogoda', 'Temperatura', 'Wiatr']
X_encoded = df[features].copy()

for col in features:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(df[col])
    le_dict[col] = le

# Kodowanie celu
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(df['Gra'])


In [12]:
# Trening modelu
model = CategoricalNB()
model.fit(X_encoded, y_encoded)


# Predykcja dla nowego rekordu
new_record = {
    'Pogoda': 'Słońce',
    'Temperatura': 'Zimno',
    'Wiatr': 'Silny'
}

new_record_encoded = []
for feature in features:
    value = new_record[feature]
    encoded_value = le_dict[feature].transform([value])[0]
    new_record_encoded.append(encoded_value)

new_record_array = np.array([new_record_encoded])

# Predykcja
prediction = model.predict(new_record_array)
probabilities = model.predict_proba(new_record_array)

print("\n" + "=" * 60)
print("WERYFIKACJA ZE SKLEARN")
print("=" * 60)
print(f"Predykcja: {le_target.inverse_transform(prediction)[0]}")
print(f"Prawdopodobieństwa: TAK={probabilities[0][1]:.3f}, NIE={probabilities[0][0]:.3f}")


WERYFIKACJA ZE SKLEARN
Predykcja: TAK
Prawdopodobieństwa: TAK=0.683, NIE=0.317


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but CategoricalNB was fitted with feature names
  warnings.warn(
